<a href="https://colab.research.google.com/github/pCodePy/gColab/blob/gColab-branch-01/pyGen_DTM_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
##### First function will take datetime series
##### and split datetime object by

##### Second function can generate series 
##### with start/end periods

# -- date;
# -- year;
# -- qtr;
# -- monNo;
# -- monName;
# -- weekNo (1, 2, ..., 52);
# -- day (1, 2, ..., 7);
# -- dayMon (day number in a month: 1, 2, ..., 31);
# -- dayYr (day number in a year: 1, 2, ..., 365);
# -- dow (day of week: Mon, Tue, ..., Sun);
# -- isWeekend (1 for weekend, 0 for weekday);
# -- dayType (1 = Mondays, 2 = Tue/Wed/Thu, ...);
# -- hour
# -- isHoliday (1 for holidays, 0 for all else) => according to BG calendar;
# -- loadType (peak [CET: 9.00-20.00 / EET: 10.00-21.00], offpeak);

In [0]:
### For holidays use the below library:
# -- check: https://pypi.org/project/workalendar/
# -- and: https://pypi.org/project/holidays/ 
# -- which is another library for holidays

# !pip install workalendar
# from datetime import date
# from workalendar.europe import Bulgaria
# cal = Bulgaria()
# cal.holidays(2019)

In [0]:
# Get time series data to test function:

# -- mount gDrive and upload test dataset
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Load libraries and read test dataset

!pip install workalendar
from workalendar.europe import Bulgaria
import numpy as np
import pandas as pd
import datetime

df = pd.read_excel('/content/drive/My Drive/py-mc-sim-dataset/ibexTimeSeries.xlsx', index_col=0)

In [4]:
# Inspect data
print(df.index)
print('----- Dataframe columns -----')
print(df.columns)
print('----- Dataframe info -----')
print(df.info())
print('----- Inspect dataset -----')
print(df.head())
print('----- Any NaN values? -----')
print(df.isnull().sum())
print('----- Summary -----')
print(round(df.describe()))

DatetimeIndex(['2017-01-01 01:00:00', '2017-01-01 02:00:00',
               '2017-01-01 03:00:00', '2017-01-01 04:00:00',
               '2017-01-01 05:00:00', '2017-01-01 06:00:00',
               '2017-01-01 07:00:00', '2017-01-01 08:00:00',
               '2017-01-01 09:00:00', '2017-01-01 10:00:00',
               ...
               '2019-03-31 14:00:00', '2019-03-31 15:00:00',
               '2019-03-31 16:00:00', '2019-03-31 17:00:00',
               '2019-03-31 18:00:00', '2019-03-31 19:00:00',
               '2019-03-31 20:00:00', '2019-03-31 21:00:00',
               '2019-03-31 22:00:00', '2019-03-31 23:00:00'],
              dtype='datetime64[ns]', name='Datetime', length=19679, freq=None)
----- Dataframe columns -----
Index(['Price'], dtype='object')
----- Dataframe info -----
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 19679 entries, 2017-01-01 01:00:00 to 2019-03-31 23:00:00
Data columns (total 1 columns):
Price    19679 non-null float64
dtypes: float64(1)
memory

In [7]:
type(df.index)
isinstance(df.index, pd.DatetimeIndex)

True

In [0]:
df = pd.read_excel('/content/drive/My Drive/py-mc-sim-dataset/ibexTimeSeries.xlsx', index_col=0)
def tsTransform(df, holCalendar=True, sumInfo=True):
    """
    df:
        takes dataframe object and checks if object is of pd.DataFrame type,
        also checks if index is of dtype: datetime;
    """
    if isinstance(df, pd.DataFrame)==True:

        if isinstance(df.index, pd.DatetimeIndex)==True:
            print('Dataframe is of the correct type \n')

        if isinstance(df.index, pd.DatetimeIndex)==False:
            print('Dataframe index is not of datetime type')
        
    else:
        print('Function expects pd.DataFrame object')
    
    # -- create necessary dictionaries:
    strList_monNo = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',
                     7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
    strList_dayWk = {0:1, 1:2, 2:3, 3:4, 4:5, 5:6, 6:7}
    strList_dow = {0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
    strList_isWknd = {'Mon':0, 'Tue':0, 'Wed':0, 'Thu':0, 'Fri':0, 'Sat':1, 'Sun':1}
    strList_DayType = {'Mon':1,'Tue':2,'Wed':2,'Thu':2,'Fri':3,'Sat':4,'Sun':5}

    # -- initiate new columns:
    df['Date'] = df.index.date
    df['Year'] = df.index.to_series().apply(lambda x: x.year)
    df['Qtr'] = df.index.to_series().apply(lambda x: x.quarter)
    df['MonNo'] = df.index.to_series().apply(lambda x: x.month)
    df['MonNm'] = df['MonNo'].map(strList_monNo)
    df['WeekNo'] = df.index.week
    df['DayYr'] = df.index.dayofyear
    df['DayMn'] = df.index.to_series().apply(lambda x: x.day)
    df['DayWk'] = df.index.to_series().apply(lambda x: x.dayofweek).map(strList_dayWk)
    df['DoW'] = df.index.to_series().apply(lambda x: x.dayofweek).map(strList_dow)
    df['DayTp'] = df['DoW'].map(strList_DayType)
    df['Hour'] = df.index.to_series().apply(lambda x: x.hour)
    df['isWknd'] = df['DoW'].map(strList_isWknd)
    df['LoadTp'] = np.select(

        [df['Hour'].between(0, 8, inclusive=True), 
         df['Hour'].between(9, 20, inclusive=True)],
        ['offpeak', 'peak'],
        default='offpeak' )
    
    ### Holiday Calendar builder:

    # -- get all years included in the df:
    yearList = df['Year'].unique().tolist()

    # -- initiate blank lists:
    colDate = []
    colHold = []

    # -- initiate loop:
    for x in yearList:
    
        hol = cal.holidays(x)

        for i in range(len(hol)):

            colDate.append(pd.to_datetime(hol[i][0]))
            colHold.append(hol[i][1])

    # -- create holiday df:
    holidaysDF = pd.DataFrame(list(zip(colDate, colHold)), columns=['Date', 'HolName'])

    ### Merge and map both dataframes:

    # -- convert to datetime for the join procedure to work:
    df['Date'] = pd.to_datetime(df['Date'])
    holidaysDF['Date'] = pd.to_datetime(holidaysDF['Date'])

    # -- reset index and add it back as pandas merge drops index after join:
    mainDF = df.reset_index().merge(holidaysDF, how='left', on='Date', left_index=True).set_index('Datetime')

    ### Generate DF based on conditional
    if sumInfo == True:

        print('---------- Dataframe columns ---------- \n')
        print(df.index)
        print('\n ---------- Dataframe columns ---------- \n')
        print(df.columns)
        print('\n ---------- Dataframe info ---------- \n')
        print(df.info())
        print('\n ---------- Inspect dataset ---------- \n')
        print(df.head())
        print('\n ---------- Any NaN values? ---------- \n')
        print(df.isnull().sum())
        print('\n ---------- Summary --------- \n')
        print(round(df.describe()))

        if holCalendar == True:
            return mainDF
        if holCalendar == False:
            return df

    else:

         if holCalendar == True:
            return mainDF
         if holCalendar == False:
            return df


In [381]:
tsTransform(df, holCalendar=False, sumInfo=True)

Dataframe is of the correct type 

---------- Dataframe columns ---------- 

DatetimeIndex(['2017-01-01 01:00:00', '2017-01-01 02:00:00',
               '2017-01-01 03:00:00', '2017-01-01 04:00:00',
               '2017-01-01 05:00:00', '2017-01-01 06:00:00',
               '2017-01-01 07:00:00', '2017-01-01 08:00:00',
               '2017-01-01 09:00:00', '2017-01-01 10:00:00',
               ...
               '2019-03-31 14:00:00', '2019-03-31 15:00:00',
               '2019-03-31 16:00:00', '2019-03-31 17:00:00',
               '2019-03-31 18:00:00', '2019-03-31 19:00:00',
               '2019-03-31 20:00:00', '2019-03-31 21:00:00',
               '2019-03-31 22:00:00', '2019-03-31 23:00:00'],
              dtype='datetime64[ns]', name='Datetime', length=19679, freq=None)

 ---------- Dataframe columns ---------- 

Index(['Price', 'Date', 'Year', 'Qtr', 'MonNo', 'MonNm', 'WeekNo', 'DayYr',
       'DayMn', 'DayWk', 'DoW', 'DayTp', 'Hour', 'isWknd', 'LoadTp'],
      dtype='object')



,Price,Date,Year,Qtr,MonNo,MonNm,WeekNo,DayYr,DayMn,DayWk,DoW,DayTp,Hour,isWknd,LoadTp
Datetime,,,,,,,,,,,,,,,
2017-01-01 01:00:00,81.81,2017-01-01,2017,1,1,Jan,52,1,1,7,Sun,5,1,1,offpeak
2017-01-01 02:00:00,44.01,2017-01-01,2017,1,1,Jan,52,1,1,7,Sun,5,2,1,offpeak
2017-01-01 03:00:00,38.08,2017-01-01,2017,1,1,Jan,52,1,1,7,Sun,5,3,1,offpeak
2017-01-01 04:00:00,36.75,2017-01-01,2017,1,1,Jan,52,1,1,7,Sun,5,4,1,offpeak
2017-01-01 05:00:00,36.91,2017-01-01,2017,1,1,Jan,52,1,1,7,Sun,5,5,1,offpeak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-31 19:00:00,74.09,2019-03-31,2019,1,3,Mar,13,90,31,7,Sun,5,19,1,peak
2019-03-31 20:00:00,76.06,2019-03-31,2019,1,3,Mar,13,90,31,7,Sun,5,20,1,peak
2019-03-31 21:00:00,74.91,2019-03-31,2019,1,3,Mar,13,90,31,7,Sun,5,21,1,offpeak


In [288]:
# -- initiate calendar variables
cal = Bulgaria()
hol = cal.holidays(2019)

yearList = [2017, 2018, 2019]

# -- initiate blank lists
colDate = []
colHold = []

# -- loop through holidays and create df
for x in yearList:

    hol = cal.holidays(x)

    for i in range(len(hol)):

        colDate.append(pd.to_datetime(hol[i][0]))
        colHold.append(hol[i][1])

holidaysDF = pd.DataFrame(list(zip(colDate, colHold)), columns=['Date', 'HolName'])
holidaysDF.to_dict('list')

{'Date': [Timestamp('2017-01-01 00:00:00'),
  Timestamp('2017-03-03 00:00:00'),
  Timestamp('2017-04-16 00:00:00'),
  Timestamp('2017-04-17 00:00:00'),
  Timestamp('2017-05-01 00:00:00'),
  Timestamp('2017-05-06 00:00:00'),
  Timestamp('2017-05-24 00:00:00'),
  Timestamp('2017-09-06 00:00:00'),
  Timestamp('2017-09-22 00:00:00'),
  Timestamp('2017-12-24 00:00:00'),
  Timestamp('2017-12-25 00:00:00'),
  Timestamp('2017-12-26 00:00:00'),
  Timestamp('2018-01-01 00:00:00'),
  Timestamp('2018-03-03 00:00:00'),
  Timestamp('2018-04-01 00:00:00'),
  Timestamp('2018-04-02 00:00:00'),
  Timestamp('2018-05-01 00:00:00'),
  Timestamp('2018-05-06 00:00:00'),
  Timestamp('2018-05-24 00:00:00'),
  Timestamp('2018-09-06 00:00:00'),
  Timestamp('2018-09-22 00:00:00'),
  Timestamp('2018-12-24 00:00:00'),
  Timestamp('2018-12-25 00:00:00'),
  Timestamp('2018-12-26 00:00:00'),
  Timestamp('2019-01-01 00:00:00'),
  Timestamp('2019-03-03 00:00:00'),
  Timestamp('2019-04-21 00:00:00'),
  Timestamp('2019-04